## Stock Price Technical Analysis & Financial Metrics for MSFT
1. Introduction & Setup
Objective: This analysis aims to evaluate historical stock price trends using quantitative technical indicators and financial metrics. We will utilize TA-Lib for signal generation (RSI, MACD) and PyNance for risk metrics.

In [1]:
import os
import sys
import warnings
import pandas as pd
import yfinance as yf  # Helper to fetch fresh data if CSV is missing

# Add 'scripts' directory to path to allow importing custom modules
sys.path.insert(0, os.path.dirname(os.getcwd()))

# Import Custom Scripts
from scripts.data_loader import DataLoader
from scripts.stock_analysis import TechnicalAnalyzer, StockVisualizer

# Configuration
warnings.filterwarnings('ignore')
DATA_PATH = '../data/yfinance_data/MSFT.csv'
TICKER = 'MSFT'

print("✅ Libraries and Custom Scripts Loaded.")

✅ Libraries and Custom Scripts Loaded.


#### 2. Data Loading and Preparation
We use the DataLoader class to get the raw stock data. Since technical analysis requires specific columns (Open, High, Low, Close, Volume), we ensure the data is formatted correctly.


In [2]:

# Ensure the file exists
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"❌ Data file not found at: {DATA_PATH}")

# Use your existing loader
loader = DataLoader()
stock_df = loader.load_data(DATA_PATH)

# Verify OHLCV data loaded correctly
print(f"Loaded Data Shape: {stock_df.shape}")
print(stock_df.head())


Loaded Data Shape: (3774, 6)
         Date      Close       High        Low       Open    Volume
0  2009-01-02  14.872976  14.924187  14.170662  14.287715  50084000
1  2009-01-05  15.011971  15.121707  14.675444  14.777866  61475200
2  2009-01-06  15.187551  15.363130  15.077815  15.180236  58083400
3  2009-01-07  14.273087  14.843718  14.251139  14.770560  72709900
4  2009-01-08  14.719351  14.770561  14.302350  14.360876  70255400


### 3. Technical Analysis with TA-Lib
In this section, we initialize the TechnicalAnalyzer class. This encapsulates the logic for applying complex mathematical indicators to the dataframe.

Indicators Applied:

SMA (Simple Moving Average): We calculate the 20-day (short-term) and 50-day (medium-term) averages to identify trend direction.

RSI (Relative Strength Index): Measures momentum to identify overbought (>70) or oversold (<30) conditions.

MACD (Moving Average Convergence Divergence): A trend-following momentum indicator that shows the relationship between two moving averages of a stock's price.

In [3]:
# Initialize the analyzer with the dataframe
analyzer = TechnicalAnalyzer(stock_df)

# Apply the indicators (SMA, RSI, MACD)
# This modifies the internal dataframe and returns the result
df_analyzed = analyzer.apply_talib_indicators()

# Preview the new technical columns
cols_to_show = ['Close', 'SMA_20', 'SMA_50', 'RSI', 'MACD']
print(df_analyzed[cols_to_show].tail())


[1] Calculating Technical Indicators (TA-Lib)...
Indicators added: SMA_20, SMA_50, RSI, MACD variables.
                 Close      SMA_20      SMA_50        RSI      MACD
Date                                                               
2023-12-22  369.767883  368.730389  355.078464  56.363342  2.805416
2023-12-26  369.846832  368.535426  356.018111  56.447998  2.767033
2023-12-27  369.264465  368.109468  356.849368  55.591261  2.658971
2023-12-28  370.458862  367.933260  357.715942  57.031664  2.639285
2023-12-29  371.209198  367.791606  358.635943  57.954296  2.653640


### 4. Financial Metrics with PyNance
We move beyond price trends to analyze risk and return using PyNance. This library is specialized for retrieving and analyzing financial data.

Metrics Calculated:

Volatility: The degree of variation of a trading price series over time.

Sharpe Ratio: The average return earned in excess of the risk-free rate per unit of volatility (a measure of risk-adjusted return).

In [4]:
# Calculate key financial metrics
financial_metrics = analyzer.calculate_financial_metrics()

# Display results in a more readable format
print("Financial Metrics Summary")
print("-" * 50)
print(f"Volatility (Annualized): {financial_metrics['volatility']:.2f}%")
print(f"Sharpe Ratio: {financial_metrics['sharpe_ratio']:.2f}")
print(f"Maximum Drawdown: {financial_metrics['max_drawdown']:.2%}")


Financial Metrics Summary
--------------------------------------------------
Volatility (Annualized): 0.27%
Sharpe Ratio: 0.90
Maximum Drawdown: -37.15%


### 5. Visualization
Finally, we use the StockVisualizer class to interpret the data visually. Charts are essential for spotting entry/exit points that raw numbers might miss.

#### A. Price Trend & Moving Averages

In [5]:
viz = StockVisualizer(df_analyzed)
viz.plot_price_and_ma()

#### B. Momentum (RSI)
This plot helps identify potential reversal points.

In [6]:
viz.plot_rsi()

#### C. Trend Strength (MACD)
This visualizes the convergence and divergence of the moving averages, providing buy/sell signals.

In [7]:
viz.plot_macd()